# LeoML

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, TextStreamer
import torch

# jphme/em_german_7b_v01
# jphme/em_german_7b_leo
model_name='jphme/em_german_mistral_v01'

model=AutoModelForCausalLM.from_pretrained(
    model_name, 
    low_cpu_mem_usage=True, 
    load_in_8bit=True
)
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id=tokenizer.eos_token_id

generation_config=GenerationConfig(
    max_new_tokens=500,
    temperature=0.4,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.2,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
    use_cache=True,
    output_attentions=False,
    output_hidden_states=False,
    output_scores=False,
    remove_invalid_values=True
)
streamer = TextStreamer(tokenizer)

# LeoLM/leo-mistral-hessianai-7b
# llm = AutoModelForCausalLM.from_pretrained(
#     pretrained_model_name_or_path="LeoLM/leo-mistral-hessianai-7b",
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
#     offload_folder="offload",
#     # use_flash_attn_2=True,  # optional
# )

RuntimeError: No GPU found. A GPU is needed for quantization.

In [ ]:
def ask_model(instruction, system='Du bist ein hilfreicher Assistent.'):
    prompt=f"{system} USER: {instruction} ASSISTANT:"
    input_tokens=tokenizer(prompt, return_tensors="pt").to(model.device)
    output_tokens=model.generate(**input_tokens,  generation_config=generation_config, streamer=streamer)[0]
    answer=tokenizer.decode(output_tokens, skip_special_tokens=True)
    return answer

In [4]:
prompt = "Schreibe einen Satz zum Theme Baurecht in Bayern."
answer=ask_model(prompt)

AttributeError: 'str' object has no attribute 'shape'